# 02 Make Network Version 01
Step 2 applies project cards, makes Python changes, and writes out a pickle of the version 01 network

In [1]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow.feather as arrow


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [3]:
# root_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'client_met_council_network')
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
input_dir = os.path.join(root_dir, 'network_standard')
pickle_dir = input_dir
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'lasso')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [4]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-20 09:36:35, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 09:36:35, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


### Read Version 00 Scenario

In [5]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

### Compute Number of Lanes 

In [6]:
v_01_scenario.road_net.links_df = v_01_scenario.road_net.links_df.drop('lanes', axis = 1)
v_01_scenario.road_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

In [7]:
v_01_scenario.road_net = metcouncil.calculate_number_of_lanes(
        roadway_net=v_01_scenario.road_net,
        parameters=parameters,
)
v_01_scenario.road_net.links_df.lanes.value_counts()

2020-10-20 09:37:57, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-20 09:37:57, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-20 09:37:57, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 09:37:57, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 09:38:18, INFO: Finished calculating number of lanes to: lanes
2020-10-20 09:38:18, INFO: Finished calculating number of lanes to: lanes


1.0    301047
2.0     26560
3.0      1734
4.0       545
5.0        66
6.0         6
Name: lanes, dtype: int64

### Write Feather

In [12]:
df = v_01_scenario.road_net.links_df.drop(['geometry', 'locationReferences'], axis = 1)
arrow.write_feather(df, os.path.join(input_dir, "link.feather"))